In [1]:
! pip install ultralytics filterpy nest_asyncio motmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 10.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.5/161.5 kB 12.9 MB/s eta 0:00:00
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=e43020d87d226c2160f307222f406a1cea8f43a0190228e26720d82bfee0455c
  Stored in directory: /root/.cache/pip/wheels/77/bf/4c/b0c3f4798a0166668752312a67118b27a3cd341e13ac0ae6ee
Successfully built filterpy


## Class: GMC (Global Motion Compensation)

**Mô tả:**
Lớp xử lý bù trừ chuyển động nền cho video (đặc biệt hữu ích cho UAV/Drone). Sử dụng phương pháp *Sparse Optical Flow* để ước lượng chuyển động của camera và căn chỉnh lại hệ tọa độ cho các đối tượng đang theo dõi.

**Phương pháp:**
1. **Downscaling:** Giảm độ phân giải ảnh để tăng tốc độ xử lý.
2. **Feature Detection:** Sử dụng thuật toán *FAST* để tìm điểm đặc trưng.
3. **Matching:** Sử dụng *Lucas-Kanade Optical Flow* để khớp điểm giữa 2 frames liên tiếp.
4. **Transformation:** Dùng *RANSAC* (trong `estimateAffinePartial2D`) để loại bỏ nhiễu và tính ma trận Affine.


In [2]:
import gradio as gr
import cv2
import numpy as np
import os
from ultralytics import YOLO
from scipy.optimize import linear_sum_assignment
from filterpy.kalman import KalmanFilter
import subprocess
import pandas as pd
import motmetrics as mm


if not hasattr(np, 'asfarray'):
    np.asfarray = lambda x: np.asarray(x, dtype=np.float64)

class GMC:
    def __init__(self, downscale=2):
        self.downscale = downscale
        self.detector = cv2.FastFeatureDetector_create(threshold=20)
        self.prev_gray = None
        self.prev_kps = None
    def apply(self, raw_frame, tracks):
        height, width = raw_frame.shape[:2]
        frame_gray = cv2.cvtColor(raw_frame, cv2.COLOR_BGR2GRAY)
        if self.downscale > 1: frame_gray = cv2.resize(frame_gray, (width // self.downscale, height // self.downscale))
        kps = self.detector.detect(frame_gray, None)
        kps = np.float32([kp.pt for kp in kps])

        if self.prev_gray is None or self.prev_kps is None or len(self.prev_kps) == 0 or len(kps) == 0:
            self.prev_gray = frame_gray; self.prev_kps = kps; return tracks
        p1, st, err = cv2.calcOpticalFlowPyrLK(self.prev_gray, frame_gray, self.prev_kps, None)


        if p1 is not None:
            status = st.flatten() == 1; good_old = self.prev_kps[status]; good_new = p1[status]
        else: good_old = np.array([]); good_new = np.array([])


        if len(good_old) < 10: self.prev_gray = frame_gray; self.prev_kps = kps; return tracks
        m, inliers = cv2.estimateAffinePartial2D(good_old, good_new)


        if m is not None:
            if self.downscale > 1: m[0, 2] *= self.downscale; m[1, 2] *= self.downscale
            for track in tracks: track.apply_gmc(m)
        self.prev_gray = frame_gray; self.prev_kps = kps
        return tracks

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


## Class: KalmanBoxTracker

**Mô tả:**
Lớp quản lý trạng thái của từng đối tượng (Track) riêng biệt bằng bộ lọc Kalman (Kalman Filter). Class này chịu trách nhiệm khởi tạo, dự đoán vị trí mới, và cập nhật vị trí khi có phát hiện (detection) mới. Đặc biệt, nó được tích hợp khả năng thích nghi với nhiễu (Adaptive Noise) và bù trừ chuyển động nền (GMC).

**Không gian trạng thái (State Space):**
Mỗi đối tượng được mô hình hóa bởi vector trạng thái 7 chiều ($dim\_x=7$):
$$\mathbf{x} = [u, v, s, r, \dot{u}, \dot{v}, \dot{s}]^T$$
Trong đó:
- $(u, v)$: Tọa độ tâm của hộp bao.
- $s$: Diện tích hộp bao ($area = w \times h$).
- $r$: Tỷ lệ khung hình ($ratio = w/h$).
- $\dot{u}, \dot{v}, \dot{s}$: Vận tốc biến thiên của tâm và diện tích.

**Các phương thức chính:**

### 1. `__init__(self, bbox)`
Khởi tạo bộ lọc Kalman cho một đối tượng mới.
- **Thiết lập ma trận:**
  - `F` (Transition Matrix): Ma trận chuyển trạng thái, mô hình hóa chuyển động với vận tốc không đổi.
  - `H` (Measurement Matrix): Ma trận đo lường, ánh xạ trạng thái sang không gian quan sát (chỉ đo được $u, v, s, r$).
  - `P, R, Q`: Các ma trận hiệp phương sai nhiễu.
- **Trạng thái:** Khởi tạo `age=0`, `hits=0`, `state=0` (Tentative - chưa xác nhận).

### 2. `apply_gmc(self, warp_matrix)`
**Chức năng quan trọng:** Cập nhật lại vị trí $(u, v)$ của đối tượng dựa trên chuyển động của Camera.
- **Input:** Ma trận biến đổi Affine $2 \times 3$ từ module GMC.
- **Logic:** Nhân vector vị trí $[u, v, 1]$ với ma trận biến đổi để "dời" đối tượng về đúng vị trí trong hệ tọa độ mới trước khi thực hiện dự đoán.

### 3. `update(self, bbox, confidence=None)`
Cập nhật trạng thái bộ lọc khi có kết quả phát hiện (Detection) mới từ YOLO.
- **Adaptive Measurement Noise (Thích nghi nhiễu):**
  Nếu có `confidence` (độ tin cậy), hệ thống sẽ tự động điều chỉnh ma trận nhiễu đo lường `R`.
  - *Confidence thấp* $\rightarrow$ Tăng `R` $\rightarrow$ Tin vào dự đoán (Predict) hơn.
  - *Confidence cao* $\rightarrow$ Giảm `R` $\rightarrow$ Tin vào phát hiện (Detection) hơn.
- **Chuyển trạng thái:** Nếu đối tượng xuất hiện liên tiếp 3 lần (`hits >= 3`), chuyển từ `Tentative` (0) sang `Confirmed` (1).

### 4. `predict(self)`
Dự đoán trạng thái ở khung hình tiếp theo.
- Tăng tuổi (`age`) của quỹ đạo.
- Nếu mất dấu (`time_since_update > 0`), reset chuỗi `hit_streak` về 0.
- Trả về hộp bao dự đoán.

### Helper Methods (Static)
- `convert_bbox_to_z(bbox)`: Chuyển đổi từ `[x1, y1, x2, y2]` sang không gian trạng thái `[u, v, s, r]`.
- `convert_x_to_bbox(x)`: Chuyển ngược lại từ không gian trạng thái về hộp bao chuẩn để hiển thị.

---


In [3]:
class KalmanBoxTracker:
    count = 0
    def __init__(self, bbox):
        self.kf = KalmanFilter(dim_x=7, dim_z=4)
        self.kf.F = np.array([[1,0,0,0,1,0,0], [0,1,0,0,0,1,0], [0,0,1,0,0,0,1],[0,0,0,1,0,0,0],  [0,0,0,0,1,0,0], [0,0,0,0,0,1,0], [0,0,0,0,0,0,1]])
        self.kf.H = np.array([[1,0,0,0,0,0,0], [0,1,0,0,0,0,0], [0,0,1,0,0,0,0], [0,0,0,1,0,0,0]])
        self.kf.R[2:,2:] *= 10.; self.kf.P[4:,4:] *= 1000.; self.kf.P *= 10.
        self.kf.Q[-1,-1] *= 0.01; self.kf.Q[4:,4:] *= 0.01
        self.kf.x[:4] = self.convert_bbox_to_z(bbox)
        self.time_since_update = 0; self.id = KalmanBoxTracker.count
        KalmanBoxTracker.count += 1; self.history = []; self.hits = 0; self.hit_streak = 0; self.age = 0; self.state = 0
    def apply_gmc(self, warp_matrix):
        pos = np.array([self.kf.x[0, 0], self.kf.x[1, 0], 1.0])
        new_pos = warp_matrix @ pos
        self.kf.x[0, 0] = new_pos[0]; self.kf.x[1, 0] = new_pos[1]
    def update(self, bbox, confidence=None):
        self.time_since_update = 0; self.history = []; self.hits += 1; self.hit_streak += 1
        if confidence:
            r_factor = (1.0 - confidence) * 20.0
            self.kf.R = np.diag([1., 1., 10., 10.]) + np.diag([r_factor, r_factor, r_factor, r_factor])
        self.kf.update(self.convert_bbox_to_z(bbox))
        if self.state == 0 and self.hits >= 3: self.state = 1
    def predict(self):
        if((self.kf.x[6, 0] + self.kf.x[2, 0]) <= 0): self.kf.x[6, 0] *= 0.0
        self.kf.predict(); self.age += 1
        if(self.time_since_update > 0): self.hit_streak = 0
        self.time_since_update += 1
        self.history.append(self.convert_x_to_bbox(self.kf.x))
        return self.history[-1]
    def get_state(self):
        return self.convert_x_to_bbox(self.kf.x)
    @staticmethod
    def convert_bbox_to_z(bbox):
        w = bbox[2] - bbox[0]; h = bbox[3] - bbox[1]; x = bbox[0] + w/2.; y = bbox[1] + h/2.
        s = w * h; r = w / float(h); return np.array([x, y, s, r]).reshape((4, 1))
    @staticmethod
    def convert_x_to_bbox(x, score=None):
        w = np.sqrt(x[2] * x[3]); h = x[2] / w
        if(score is None): return np.array([x[0]-w/2., x[1]-h/2., x[0]+w/2., x[1]+h/2.]).reshape((1,4))
        else: return np.array([x[0]-w/2., x[1]-h/2., x[0]+w/2., x[1]+h/2., score]).reshape((1,5))


## Module: Data Association & Utilities

Module này cung cấp các công cụ toán học và xử lý dữ liệu cần thiết để liên kết các phát hiện (detections) với các quỹ đạo (trackers).

### 1. Hàm `iou_batch(bb_test, bb_gt)`
**Mô tả:**
Tính toán chỉ số IoU (Intersection over Union) giữa hai tập hợp các hộp bao (Bounding Boxes) một cách song song (vectorized operation) sử dụng Numpy broadcasting. Đây là phiên bản tối ưu tốc độ cao thay vì dùng vòng lặp `for`.

**Tham số:**
- `bb_test`: Mảng Numpy `(N, 4)` chứa các hộp bao dự đoán (Detections).
- `bb_gt`: Mảng Numpy `(M, 4)` chứa các hộp bao thực tế (Trackers).

**Cơ chế:**
Sử dụng kỹ thuật `expand_dims` để tạo ma trận so sánh kích thước `(N, M)`, cho phép tính toán đồng thời diện tích giao (Intersection) và diện tích hợp (Union) của tất cả các cặp có thể.

**Trả về:**
- Ma trận `(N, M)` chứa giá trị IoU (từ 0.0 đến 1.0) của từng cặp.

---

### 2. Hàm `associate_detections_to_trackers`
**Mô tả:**
Thực hiện ghép nối (Matching) giữa các phát hiện mới và các quỹ đạo hiện có dựa trên ma trận IoU.

**Thuật toán:**
1. **Tính Ma trận IoU:** Gọi `iou_batch` để lấy độ chồng lấn giữa mọi cặp Detection-Tracker.
2. **Hungarian Algorithm:**
   - Nếu ma trận đơn giản (mỗi hàng/cột chỉ có 1 giá trị vượt ngưỡng), thực hiện ghép trực tiếp.
   - Nếu phức tạp (xung đột), sử dụng thuật toán `linear_sum_assignment` (Hungarian) từ thư viện `scipy` để tìm phương án ghép cặp tối ưu (tổng IoU lớn nhất).
3. **Lọc theo ngưỡng (Thresholding):**
   - Loại bỏ các cặp ghép có `IoU < iou_threshold` (mặc định 0.3). Coi như không khớp.
4. **Phân loại đầu ra:**
   - `matches`: Các cặp đã ghép thành công.
   - `unmatched_detections`: Các phát hiện mới không khớp với ai (sẽ tạo Track mới).
   - `unmatched_trackers`: Các quỹ đạo cũ không tìm thấy đối tượng (có thể đã bị mất dấu).

**Tham số:**
- `detections`: Danh sách hộp bao từ YOLO.
- `trackers`: Danh sách hộp bao dự đoán từ Kalman Filter.
- `iou_threshold`: Ngưỡng chồng lấn tối thiểu để chấp nhận ghép cặp (Mặc định: 0.3).

---

### 3. Hàm `load_mot_gt(gt_path)`
**Mô tả:**
Đọc và phân tích file dữ liệu Ground Truth (GT) theo chuẩn MOTChallenge (MOT15/16/17/20) để phục vụ việc đánh giá.

**Định dạng hỗ trợ:**
Hỗ trợ cả file CSV (phân cách bằng dấu phẩy) và file text phân cách bằng khoảng trắng. Cấu trúc dòng tiêu chuẩn:
`frame, id, bb_left, bb_top, bb_width, bb_height, conf, x, y, z`

**Xử lý:**
- Tự động phát hiện định dạng phân cách (phẩy hoặc khoảng trắng).
- Chuyển đổi tọa độ từ `[x, y, w, h]` (chuẩn MOT) sang `[x1, y1, x2, y2]` (chuẩn xử lý nội bộ).
- Gom nhóm dữ liệu theo từng `frame` để truy xuất nhanh (`dict key = frame_id`).

**Trả về:**
- `gt_dict`: Dictionary dạng `{frame_id: [[x1, y1, x2, y2, obj_id], ...]}`.

In [4]:
def iou_batch(bb_test, bb_gt):
    bb_gt = np.expand_dims(bb_gt, 0); bb_test = np.expand_dims(bb_test, 1)
    xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0]); yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
    xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2]); yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
    w = np.maximum(0., xx2 - xx1); h = np.maximum(0., yy2 - yy1); wh = w * h
    o = wh / ((bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1]) + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1]) - wh)
    return(o)

def associate_detections_to_trackers(detections, trackers, iou_threshold=0.3):
    if(len(trackers)==0): return np.empty((0,2),dtype=int), np.arange(len(detections)), np.empty((0,),dtype=int)
    iou_matrix = iou_batch(detections, trackers)
    if min(iou_matrix.shape) > 0:
        a = (iou_matrix > iou_threshold).astype(np.int32)
        if a.sum(1).max() == 1 and a.sum(0).max() == 1: matched_indices = np.stack(np.where(a), axis=1)
        else: row_ind, col_ind = linear_sum_assignment(-iou_matrix); matched_indices = np.stack((row_ind, col_ind), axis=1)
    else: matched_indices = np.empty((0,2),dtype=int)
    unmatched_detections = [d for d, det in enumerate(detections) if d not in matched_indices[:,0]]
    unmatched_trackers = [t for t, trk in enumerate(trackers) if t not in matched_indices[:,1]]
    matches = []
    for m in matched_indices:
        if(iou_matrix[m[0], m[1]] < iou_threshold): unmatched_detections.append(m[0]); unmatched_trackers.append(m[1])
        else: matches.append(m.reshape(1,2))
    if(len(matches)==0): matches = np.empty((0,2),dtype=int)
    else: matches = np.concatenate(matches, axis=0)
    return matches, np.array(unmatched_detections, dtype=int), np.array(unmatched_trackers, dtype=int)

def load_mot_gt(gt_path):
    if gt_path is None: return {}
    if hasattr(gt_path, 'name'): gt_path = gt_path.name
    gt_dict = {}
    try:
        try: df = pd.read_csv(gt_path, header=None, sep=',')
        except: df = pd.read_csv(gt_path, header=None, sep=r'\s+', engine='python')
        for index, row in df.iterrows():
            try:
                if not str(row[0]).replace('.','',1).isdigit(): continue
                frame = int(float(row[0])); obj_id = int(float(row[1]))
                x1, y1, w, h = float(row[2]), float(row[3]), float(row[4]), float(row[5])
                if frame not in gt_dict: gt_dict[frame] = []
                gt_dict[frame].append([x1, y1, x1+w, y1+h, obj_id])
            except ValueError: continue
    except Exception as e: print(f"GT Error: {e}"); return {}
    return gt_dict

In [7]:
MODEL_OPTIONS = {
    "EfficientNetB0": "/content/drive/MyDrive/tracking/efficientnetB0-yolov8.pt",
    "EfficientNetB3": "/content/drive/MyDrive/tracking/efficientnetB0-yolov8.pt",
    "MobileNet": "yolov8s.pt",
    "ConvNext-T": "/content/drive/MyDrive/tracking/convnext_T_best.pt",
    "ConvNext-S": "/content/drive/MyDrive/tracking/convnext_S_best.pt"
}


def detect_objects_frame_1(video_path, model_name):
    """
    Chạy YOLO trên frame 1, cắt ảnh các đối tượng detected.
    Trả về: List ảnh (cho Gallery) và List bbox (lưu vào State để dùng sau).
    """
    if video_path is None: return [], [], []

    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    cap.release()

    if not ret: return [], [], []

    # Load model
    model_path = MODEL_OPTIONS.get(model_name, "yolov8n.pt")
    try: model = YOLO(model_path)
    except: model = YOLO("yolov8n.pt")

    # Detect
    results = model(frame, verbose=False, iou=0.45, conf=0.1)[0]

    gallery_images = []
    detected_boxes = [] # Lưu coordinate thực tế [x1, y1, x2, y2]

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    if results.boxes:
        for i, box in enumerate(results.boxes):
            x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
            crop = frame_rgb[y1:y2, x1:x2]
            gallery_images.append((crop, f"Object {i}"))
            detected_boxes.append([x1, y1, x2, y2])

    # Reset selected indices state khi scan mới
    return gallery_images, detected_boxes, []

def on_select_object(evt: gr.SelectData, detected_boxes, current_selection_indices):
    """
    Hàm Toggle chọn nhiều đối tượng.
    """
    if current_selection_indices is None:
        current_selection_indices = []

    index = evt.index

    # Toggle logic: Nếu đã có thì xóa, chưa có thì thêm
    if index in current_selection_indices:
        current_selection_indices.remove(index)
    else:
        current_selection_indices.append(index)

    # Sắp xếp lại cho đẹp
    current_selection_indices.sort()

    # Lấy danh sách các box tương ứng
    selected_boxes = [detected_boxes[i] for i in current_selection_indices if i < len(detected_boxes)]

    feedback_str = f"Đang chọn các Object: {current_selection_indices}" if current_selection_indices else "Chưa chọn đối tượng nào (Sẽ track tất cả)"

    return feedback_str, current_selection_indices, selected_boxes

def clear_selection():
    return "Đã xóa chọn. Tracking tất cả.", [], []

def calculate_iou_single(box1, box2):
    # box: [x1, y1, x2, y2]
    xx1 = max(box1[0], box2[0]); yy1 = max(box1[1], box2[1])
    xx2 = min(box1[2], box2[2]); yy2 = min(box1[3], box2[3])
    w = max(0, xx2 - xx1); h = max(0, yy2 - yy1)
    inter = w * h
    area1 = (box1[2]-box1[0])*(box1[3]-box1[1])
    area2 = (box2[2]-box2[0])*(box2[3]-box2[1])
    union = area1 + area2 - inter
    return inter/union if union > 0 else 0

In [ ]:
def run_tracking_demo(video_path, gt_path, model_selection, conf_threshold, iou_threshold, target_boxes_list, progress=gr.Progress()):
    if video_path is None: return None, "Vui lòng upload video."

    model_path = MODEL_OPTIONS.get(model_selection, model_selection)

    target_track_ids = set()
    is_selective_mode = target_boxes_list is not None and len(target_boxes_list) > 0

    if is_selective_mode:
        print(f"🎯 Số lượng targets cần khóa: {len(target_boxes_list)}")

    try: model = YOLO(model_path)
    except: model = YOLO("yolov8n.pt")

    cap = cv2.VideoCapture(video_path)
    width, height = int(cap.get(3)), int(cap.get(4))
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps < 1: fps = 30.0
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    output_path = "temp_output.mp4"
    final_output_path = "result_video.mp4"
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    gt_data = load_mot_gt(gt_path); has_gt = len(gt_data) > 0
    acc = mm.MOTAccumulator(auto_id=True)
    trackers = []; gmc = GMC(downscale=2); KalmanBoxTracker.count = 0
    frame_idx = 0

    while True:
        ret, frame = cap.read()
        if not ret: break
        frame_idx += 1
        if frame_idx % 10 == 0: progress(frame_idx / total_frames, desc=f"Processing {frame_idx}/{total_frames}")

        results = model(frame, verbose=False, iou=0.45, conf=0.1)[0]
        dets = []
        if results.boxes:
             for box in results.boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                score = float(box.conf[0].cpu().numpy())
                dets.append([x1, y1, x2, y2, score])
        dets = np.array(dets) if len(dets) > 0 else np.empty((0, 5))

        gmc.apply(frame, trackers)

        trks = np.zeros((len(trackers), 5)); to_del = []
        for t, trk in enumerate(trks):
            pos = trackers[t].predict()[0]
            trk[:] = [pos[0], pos[1], pos[2], pos[3], 0]
            if np.any(np.isnan(pos)): to_del.append(t)
        trks = np.ma.compress_rows(np.ma.masked_invalid(trks))
        for t in reversed(to_del): trackers.pop(t)

        if len(dets) > 0:
            inds_high = dets[:, 4] >= conf_threshold
            inds_low = (dets[:, 4] > 0.1) & (dets[:, 4] < conf_threshold)
            dets_high = dets[inds_high]; dets_low = dets[inds_low]
        else: dets_high = np.empty((0, 5)); dets_low = np.empty((0, 5))

        matched, unmatched_dets, unmatched_trks = associate_detections_to_trackers(dets_high, trks, iou_threshold)

        trks_remain = trks[unmatched_trks]; dets_remain = dets_low
        if len(trks_remain) > 0 and len(dets_remain) > 0:
            matched_l, _, _ = associate_detections_to_trackers(dets_remain, trks_remain, 0.1)
            for m in matched_l: trackers[unmatched_trks[m[1]]].update(dets_remain[m[0]][:4], dets_remain[m[0]][4])
        for m in matched: trackers[m[1]].update(dets_high[m[0]][:4], dets_high[m[0]][4])
        for i in unmatched_dets: trackers.append(KalmanBoxTracker(dets_high[i][:4]))

        i = len(trackers); ret_trackers = []
        for trk in reversed(trackers):
            d = trk.get_state()[0]
            if (trk.time_since_update < 1) and (trk.hit_streak >= 3 or frame_idx <= 3):
                ret_trackers.append(np.concatenate((d,[trk.id])).reshape(1,-1))
            i -= 1
            if(trk.time_since_update > 30): trackers.pop(i)

        if frame_idx == 1 and is_selective_mode and len(ret_trackers) > 0:
            for target_box in target_boxes_list:
                best_iou = 0
                best_id = -1
                for trk_data in ret_trackers:
                    d = trk_data[0]
                    trk_box = [d[0], d[1], d[2], d[3]]
                    iou = calculate_iou_single(target_box, trk_box)
                    if iou > best_iou:
                        best_iou = iou
                        best_id = int(d[4])

                if best_iou > 0.5:
                    target_track_ids.add(best_id)

            print(f"🎯 Đã khóa các ID: {target_track_ids}")

            if len(target_track_ids) == 0:
                print("⚠️ Không khớp được ID nào. Chuyển sang track all.")
                is_selective_mode = False

        if has_gt:
            t_ids = []; t_boxes = []
            for trk_data in ret_trackers:
                d = trk_data[0]
                t_ids.append(int(d[4]))
                t_boxes.append([d[0], d[1], d[2]-d[0], d[3]-d[1]])
            g_ids = []; g_boxes = []
            if frame_idx in gt_data:
                for item in gt_data[frame_idx]:
                    g_ids.append(int(item[4]))
                    g_boxes.append([item[0], item[1], item[2]-item[0], item[3]-item[1]])
            dist = mm.distances.iou_matrix(g_boxes, t_boxes, max_iou=0.5) if (len(g_boxes)>0 and len(t_boxes)>0) else []
            acc.update(g_ids, t_ids, dist)

        # --- Draw ---
        for d in ret_trackers:
            d = d[0]
            x1, y1, x2, y2, tid = int(d[0]), int(d[1]), int(d[2]), int(d[3]), int(d[4])

            should_draw = True
            color = (0, 255, 0)
            thickness = 2

            if is_selective_mode:
                if tid in target_track_ids:
                    color = (0, 0, 255)
                    thickness = 3
                else:
                    should_draw = False

            if should_draw:
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)
                label = f"TARGET {tid}" if (is_selective_mode and tid in target_track_ids) else f"ID {tid}"
                cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        out.write(frame)

    cap.release(); out.release()

    metrics_str = "Metrics:"
    if has_gt:
        mh = mm.metrics.create()
        try:
            summary = mh.compute(acc, metrics=['num_frames', 'mota', 'motp', 'idf1', 'mostly_tracked', 'mostly_lost', 'num_switches'], name='acc')
            metrics_str = mm.io.render_summary(summary, formatters=mh.formatters, namemap={'num_frames': 'Frames', 'mota': 'MOTA', 'motp': 'MOTP', 'idf1': 'IDF1', 'mostly_tracked': 'MT', 'mostly_lost': 'ML', 'num_switches': 'ID Sw'})
        except: metrics_str = "Error calculating metrics"

    if os.path.exists(final_output_path): os.remove(final_output_path)
    try: subprocess.call(args=f"ffmpeg -y -i {output_path} -c:v libx264 {final_output_path} -loglevel quiet", shell=True)
    except: final_output_path = output_path
    return final_output_path, metrics_str


with gr.Blocks(title="UAV Tracking System") as demo:
    gr.Markdown("# UAV Multi-Target Tracking")
    gr.Markdown("Upload video -> Quét -> Click chọn nhiều đối tượng (Click lần nữa để bỏ chọn) -> Start.")

    # 1. Danh sách tất cả box detect được ở frame 1
    detected_boxes_state = gr.State([])
    # 2. Danh sách index của các object đã click chọn (VD: [0, 3, 5])
    selected_indices_state = gr.State([])
    # 3. Danh sách coordinate thực tế của các object đã chọn (Output cho model)
    selected_boxes_state = gr.State([])

    with gr.Row():
        with gr.Column(scale=1):
            input_video = gr.Video(label="1. Upload Video")
            model_dd = gr.Dropdown(choices=list(MODEL_OPTIONS.keys()), value="EfficientNetB0", label="Model")

            with gr.Row():
                btn_scan = gr.Button("Quét đối tượng", variant="secondary")
                btn_clear = gr.Button("Xóa chọn", variant="stop")

            gr.Markdown("### 3. Gallery (Click để Chọn/Bỏ chọn):")
            gallery = gr.Gallery(
                label="Danh sách đối tượng",
                show_label=True,
                elem_id="gallery",
                columns=4, rows=2, height="auto",
                object_fit="contain",
                allow_preview=False
            )
            selection_info = gr.Textbox(label="Trạng thái chọn", value="Chưa chọn (Track All)", interactive=False)

            with gr.Row():
                conf_slide = gr.Slider(0.1, 0.9, 0.5, label="Conf Threshold")
                iou_slide = gr.Slider(0.1, 0.9, 0.2, label="IoU Threshold")

            input_gt = gr.File(label="Upload GT (.txt)")
            btn_run = gr.Button("START TRACKING", variant="primary")

        with gr.Column(scale=1):
            output_video = gr.Video(label="Kết quả Tracking")
            output_metrics = gr.Textbox(label="Metrics Report", lines=10)


    btn_scan.click(
        fn=detect_objects_frame_1,
        inputs=[input_video, model_dd],
        outputs=[gallery, detected_boxes_state, selected_indices_state]
    )

    gallery.select(
        fn=on_select_object,
        inputs=[detected_boxes_state, selected_indices_state],
        outputs=[selection_info, selected_indices_state, selected_boxes_state]
    )

    btn_clear.click(
        fn=clear_selection,
        inputs=[],
        outputs=[selection_info, selected_indices_state, selected_boxes_state]
    )

    btn_run.click(
        fn=run_tracking_demo,
        inputs=[input_video, input_gt, model_dd, conf_slide, iou_slide, selected_boxes_state],
        outputs=[output_video, output_metrics]
    )

if __name__ == "__main__":
    demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://001cb474ec5ef41e5d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/protocols/http/h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/middleware/proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/fastapi/applications.py", line 1139, in __call__
    await super().__call__(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/applications.py", line 107, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py", line 186, in __call__
    raise exc
  File "/usr/local/lib/python3.12/dist-packages/starlette/middleware/errors.py",

🎯 Số lượng targets cần khóa: 1
🎯 Đã khóa các ID: {0}
